In [2]:
import pandas as pd

# Load the dataset
file_path = 'gwas_celltype_filterted.csv'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()


,Genename,X,GENE,CHR,START,STOP,NSNPS,NPARAM,N,ZSTAT,P,X.1,cell_type,p_val,avg_log2FC,pct.1,pct.2,p_val_adj
0,ABAT,1,18,16,8733444,8888432,1107,105,455258,2.2912,0.010975,NaN,macrophage,5.023350e-01,0.574805,0.200,0.000,1.000000
1,ABAT,1,18,16,8733444,8888432,1107,105,455258,2.2912,0.010975,NaN,hybrid,2.232125e-03,-0.325814,0.197,0.319,1.000000
2,ABAT,1,18,16,8733444,8888432,1107,105,455258,2.2912,0.010975,NaN,astrocytes,8.295054e-08,0.278849,0.405,0.293,0.002272
3,ABHD10,576,55347,3,111662723,111722215,310,28,455258,2.2947,0.010876,NaN,macrophage,5.605091e-01,0.780858,0.400,0.250,1.000000
4,ABHD10,576,55347,3,111662723,111722215,310,28,455258,2.2947,0.010876,NaN,endothelial,1.885258e-01,-0.648312,0.000,0.214,1.000000


In [9]:
# Define thresholds for filtering
p_val_adj_threshold = 0.05
P =0.05

# Filter the data for significant differential expression
significant_genes = data[(data['p_val_adj'] < p_val_adj_threshold) & 
                         (data['P'].abs() < P)]

# Display the first few rows of the filtered data
significant_genes.head()
# Group and summarize the adjusted results by cell type
adjusted_summary_by_cell_type = significant_genes.groupby('cell_type').agg(
    Num_Significant_Genes=('Genename', 'nunique'),
    Avg_Log2FC=('avg_log2FC', 'mean'),
    Min_GWAS_P=('P', 'min'),
    Avg_ZSTAT=('ZSTAT', 'mean')
).reset_index()
adjusted_summary_by_cell_type

,cell_type,Num_Significant_Genes,Avg_Log2FC,Min_GWAS_P,Avg_ZSTAT
0,astrocytes,26,-0.023571,1.927900e-13,3.194850
1,microglia,25,0.280112,1.736400e-13,3.542232
2,neurons,11,0.003435,2.492200e-05,2.666955
3,oligodendrocytes,33,0.135646,1.530900e-10,2.829148
4,opc,13,0.170859,1.927900e-13,3.017162


In [3]:
# Adjusting the thresholds
adjusted_p_val_adj_threshold = 0.1  # Less stringent p-value threshold
adjusted_log2FC_threshold = 0.5     # Lower fold change threshold

# Re-filter the data with adjusted thresholds
adjusted_significant_genes = data[(data['p_val_adj'] < adjusted_p_val_adj_threshold) & 
                                  (data['avg_log2FC'].abs() > adjusted_log2FC_threshold)]

# Group and summarize the adjusted results by cell type
adjusted_summary_by_cell_type = adjusted_significant_genes.groupby('cell_type').agg(
    Num_Significant_Genes=('Genename', 'nunique'),
    Avg_Log2FC=('avg_log2FC', 'mean'),
    Min_GWAS_P=('P', 'min'),
    Avg_ZSTAT=('ZSTAT', 'mean')
).reset_index()

# Sorting by the number of significant genes for better interpretation
adjusted_summary_sorted = adjusted_summary_by_cell_type.sort_values(by='Num_Significant_Genes', ascending=False)

# Display the adjusted summary
adjusted_summary_sorted.head()


,cell_type,Num_Significant_Genes,Avg_Log2FC,Min_GWAS_P,Avg_ZSTAT
1,microglia,11,0.408751,7.792700e-07,2.887309
2,oligodendrocytes,11,0.449586,1.030600e-06,3.000018
0,astrocytes,5,-0.163080,1.927900e-13,3.574520
3,opc,4,0.760417,2.492200e-05,3.000350


In [4]:
# Extracting unique gene names and their corresponding GENE IDs for each cell type
significant_genes_ids_by_cell_type = adjusted_significant_genes.groupby('cell_type').apply(
    lambda x: x[['Genename', 'GENE']].drop_duplicates().reset_index(drop=True)
).reset_index().drop(columns='level_1')

# Renaming columns for clarity
significant_genes_ids_by_cell_type.columns = ['Cell Type', 'Gene Name', 'GENE ID']

# Display the list of significant genes with their GENE IDs by cell type
significant_genes_ids_by_cell_type.pivot(index='GENE ID', columns='Cell Type', values='Gene Name')


Cell Type,astrocytes,microglia,oligodendrocytes,opc
GENE ID,,,,
348,APOE,NaN,NaN,NaN
558,NaN,AXL,NaN,NaN
563,NaN,NaN,AZGP1,NaN
1956,NaN,NaN,NaN,EGFR
2180,NaN,ACSL1,NaN,NaN
2289,FKBP5,FKBP5,FKBP5,FKBP5
3696,ITGB8,NaN,NaN,NaN
4643,NaN,MYO1E,NaN,NaN
5166,NaN,PDK4,PDK4,NaN


In [6]:
import csv

# Function to process the GWAS file and extract rsIDs for given GENE IDs
def extract_rsids_for_genes(gwas_file_path, gene_ids):
    rsid_mapping = {}
    with open(gwas_file_path, 'r') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            # Skip header or metadata lines
            if row[0].startswith('#'):
                continue
            try:
                gene_id = int(row[0])  # Assuming the first element is the GENE ID
                rsids = row[2:]        # rsIDs start from the third element
                if gene_id in gene_ids:
                    rsid_mapping[gene_id] = rsids
            except ValueError:
                # Handle lines that don't start with a GENE ID integer
                continue
    return rsid_mapping

# Extracting unique GENE IDs from the significant genes data
unique_gene_ids = set(adjusted_significant_genes['GENE'].unique())

# Extract rsIDs for these GENE IDs
gene_to_rsid_mapping = extract_rsids_for_genes("/MAGMA_Files/GSE188545_RAW.35UP.10DOWN/gwas_Alz.tsv.35UP.10DOWN.genes.annot", unique_gene_ids)

# Checking a small part of the mapping to verify
list(gene_to_rsid_mapping.items())[:2]  # Displaying first two mappings as an example


[(50809,
  ['rs12743389',
   'rs78491441',
   'rs74059000',
   'rs61778528',
   'rs951805',
   'rs192201838',
   'rs528618788',
   'rs111754764',
   'rs12754117',
   'rs74549902',
   'rs76504397',
   'rs710311',
   'rs35933697',
   'rs72650888',
   'rs529162',
   'rs76739844',
   'rs12723152',
   'rs1888759',
   'rs112019559',
   'rs6670897',
   'rs186602606',
   'rs6668591',
   'rs6679753',
   'rs76722220',
   'rs12123092',
   'rs12130841',
   'rs541464',
   'rs11802176',
   'rs72650890',
   'rs17408019',
   'rs12121807',
   'rs17408061',
   'rs143662289',
   'rs17448349',
   'rs2274119',
   'rs667687',
   'rs148409310',
   'rs76336624',
   'rs12404954',
   'rs140054783',
   'rs12724604',
   'rs115766888',
   'rs147651581',
   'rs56356353',
   'rs145914118',
   'rs12729575',
   'rs12407704',
   'rs80124376',
   'rs77401417',
   'rs114080907',
   'rs114631712',
   'rs76505777',
   'rs188163659',
   'rs76298514',
   'rs142791317',
   'rs12126786',
   'rs658851',
   'rs12137996',
   'rs3

In [8]:
# Function to create a file for each cell type with gene names, GENE IDs, and corresponding rsIDs
def create_file_for_cell_type(cell_type, cell_type_genes, gene_to_rsid_mapping, directory):
    filename = f"{directory}/{cell_type}_genes_rsids.csv"
    with open(filename, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Gene Name', 'GENE ID', 'rsIDs'])
        for gene_name, gene_id in cell_type_genes:
            rsids = gene_to_rsid_mapping.get(gene_id, ['No rsIDs found'])
            writer.writerow([gene_name, gene_id, '; '.join(rsids)])
    return filename

# Directory for saving files
output_directory = '/Downloads/cell_rsids'

# Generating files for each cell type
files = {}
for cell_type in significant_genes_ids_by_cell_type['Cell Type'].unique():
    cell_type_genes = significant_genes_ids_by_cell_type[
        significant_genes_ids_by_cell_type['Cell Type'] == cell_type
    ][['Gene Name', 'GENE ID']].values

    file_path = create_file_for_cell_type(cell_type, cell_type_genes, gene_to_rsid_mapping, output_directory)
    files[cell_type] = file_path

files


{'astrocytes': '/Users/simranjitvirk/Downloads/cell_rsids/astrocytes_genes_rsids.csv',
 'microglia': '/Users/simranjitvirk/Downloads/cell_rsids/microglia_genes_rsids.csv',
 'oligodendrocytes': '/Users/simranjitvirk/Downloads/cell_rsids/oligodendrocytes_genes_rsids.csv',
 'opc': '/Users/simranjitvirk/Downloads/cell_rsids/opc_genes_rsids.csv'}